In [1]:
from torch.utils.data import DataLoader
import utils
config = {}
config['hidden_dims'] = [16]
config['num_layers'] = len(config['hidden_dims']) + 1
config['dataset_folder'] = '../tmp/'
dataset_args = ('train', config['num_layers'])

datasets = utils.get_dataset_gcn(dataset_args, 'tmp/', is_debug=False)
loaders = []
for i in range(len(datasets)):
    loaders.append(DataLoader(dataset=datasets[i], batch_size=4,
                    shuffle=True, collate_fn=datasets[i].collate_wrapper))


52
--------------------------------
Reading edge dataset from Train\HE_T1087_84_Default_Extended_1_3
Dropping edge (1,24)
Dropping edge (1,150)
Dropping edge (24,134)
Dropping edge (71,122)
Dropping edge (71,148)
Dropping edge (101,106)
Dropping edge (101,190)
Dropping edge (106,109)
Dropping edge (109,134)
Dropping edge (122,209)
Dropping edge (148,151)
Dropping edge (150,151)
Dropping edge (188,189)
Dropping edge (188,229)
Dropping edge (189,192)
Dropping edge (190,191)
Dropping edge (191,192)
Dropping edge (209,225)
------ Removing outer triangles -------
------ Removing outer triangles -------
------ Removing outer triangles -------
Dropping edge (224,225)
Dropping edge (224,227)
Dropping edge (227,228)
Dropping edge (228,229)
Finished reading data.
Setting up graph.
self.features.shape: torch.Size([242, 4])
Finished setting up graph.
Setting up examples.
Finished setting up examples.
Dataset properties:
Mode: train
Number of vertices: 242
Number of edges: 679
Number of triangles: 

In [ ]:
for (idx, batch) in enumerate(loaders[i]):
    adj, features, edge_features, edges, labels, dist, triangles = batch
               

In [3]:
adj.shape, features.shape, edge_features.shape, edges.shape, labels.shape, dist.shape, len(triangles)

(torch.Size([1, 1325, 1325]),
 torch.Size([1325, 4]),
 torch.Size([4, 1325, 1325]),
 (4, 2),
 torch.Size([4]),
 torch.Size([1, 1325, 1325]),
 3921)

{(752, 755, 761),
 (421, 917, 918),
 (909, 910, 911),
 (974, 982, 985),
 (98, 99, 487),
 (791, 880, 881),
 (457, 458, 722),
 (538, 539, 540),
 (234, 527, 528),
 (336, 355, 1236),
 (1215, 1216, 1219),
 (1134, 1136, 1137),
 (507, 508, 600),
 (431, 977, 979),
 (362, 1113, 1115),
 (382, 1092, 1093),
 (576, 578, 1324),
 (425, 904, 912),
 (670, 680, 681),
 (840, 842, 844),
 (88, 91, 493),
 (1258, 1259, 1271),
 (283, 356, 1136),
 (1153, 1155, 1156),
 (996, 997, 999),
 (1117, 1118, 1120),
 (970, 976, 977),
 (217, 527, 528),
 (426, 931, 932),
 (863, 864, 865),
 (102, 103, 110),
 (249, 990, 1031),
 (792, 793, 795),
 (233, 234, 258),
 (90, 140, 1205),
 (217, 528, 529),
 (238, 249, 260),
 (55, 56, 711),
 (869, 870, 879),
 (64, 155, 648),
 (541, 544, 857),
 (737, 738, 741),
 (14, 834, 835),
 (83, 84, 1232),
 (675, 676, 677),
 (1075, 1090, 1091),
 (537, 923, 924),
 (57, 660, 711),
 (1097, 1104, 1105),
 (230, 434, 529),
 (1158, 1159, 1160),
 (397, 401, 1049),
 (449, 811, 817),
 (620, 625, 1087),
 (12

In [10]:
list( dict.fromkeys(triangles.get((312,0))) )

TypeError: 'NoneType' object is not iterable

In [4]:
for u, v in edges:
    triangle_list = triangles.get((u, v), []) or triangles.get((v, u), [])
    if len(triangle_list) == 2: # if there are two triangles
        z, w = triangle_list
    elif len(triangle_list) == 1: # if there is only one triangle
        z = triangle_list[0]
        w = z # make w=z in case of only one triangle
    else: # if there is no triangle
        continue

    print(u, v, z, w)

In [ ]:
node_features_difference = features.unsqueeze(1) - features.unsqueeze(0)
node_features_difference.shape

In [ ]:
edge_features_transposed = edge_features.permute(1, 2, 0)
edge_features_transposed.shape  

In [ ]:
combined_features = node_features_difference + edge_features_transposed
combined_features.shape

In [ ]:
import torch

aggregated_triangle_features = torch.zeros(features.shape[0], features.shape[1])

for u, v, w in triangles:
    # Extract combined features for each pair of nodes in the triangle
    combined_features_uv = combined_features[u, v]
    combined_features_vw = combined_features[v, w]
    combined_features_wu = combined_features[w, u]

    # Aggregate the combined features of each pair
    triangle_features = combined_features_uv + combined_features_vw + combined_features_wu

    # Update the aggregated triangle features for each node in the triangle
    aggregated_triangle_features[u] += triangle_features
    aggregated_triangle_features[v] += triangle_features
    aggregated_triangle_features[w] += triangle_features

aggregated_triangle_features.shape

In [ ]:
triangle_count = torch.zeros(features.shape[0])
for u, v, w in triangles:
    triangle_count[u] += 1
    triangle_count[v] += 1
    triangle_count[w] += 1

normalized_triangle_features = aggregated_triangle_features / triangle_count.unsqueeze(1)
normalized_triangle_features

In [ ]:
aggregated_triangle_features 

In [ ]:
import torch

torch.matmul(features, triangles).shape

In [ ]:
def sparse_triangle_matmul(triangle_sparse, features):
    indices = triangle_sparse._indices()
    values = triangle_sparse._values()
    size = triangle_sparse.size()
    N = size[1]
    F = features.size(1)

    output = torch.zeros((N, F))

    for i in range(indices.size(1)):
        r, n1, n2, n3 = indices[:, i]
        value = values[i]
        output[n1] += value * (features[n2] * features[n3])

    return output 

In [ ]:
import torch


if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
config['device'] = device

In [ ]:
from models import DGNN, AAGNN, EGNNC, TEGNNC

input_dim, hidden_dim,  output_dim = datasets[0].get_dims()[0], config['hidden_dims'][0], 16


channel_dim = datasets[0].get_channel()
model = TEGNNC(input_dim, hidden_dim, output_dim,
                    channel_dim, 0, config['device'])
model.to(config['device'])


In [ ]:
features, edge_features, triangles = features.to(device), edge_features.to(device), triangles
out = model(features, edge_features, triangles)

In [ ]:
# import Parameter
import torch.nn as nn
from torch.nn import Parameter

weight2 = Parameter(torch.FloatTensor(input_dim, hidden_dim))

In [ ]:
support2 = torch.matmul(features, weight2)
support1 = torch.matmul(features, weight2)

#x = torch.matmul(edge_features, support1) + support0

        # Aggregate triangle features
triangle_agg = torch.zeros_like(edge_features)
for t in triangles:
    triangle_agg[:, t[0], t[1]] += edge_features[:, t[1], t[2]] + edge_features[:, t[2], t[1]]
    triangle_agg[:, t[1], t[0]] += edge_features[:, t[0], t[2]] + edge_features[:, t[2], t[0]]
    triangle_agg[:, t[2], t[0]] += edge_features[:, t[0], t[1]] + edge_features[:, t[1], t[0]]
    # 
H_k = torch.matmul(edge_features, support1) 
#x = torch.matmul(triangle_agg, support2)

In [ ]:
x = torch.matmul(triangle_agg, support2) + torch.matmul(edge_features, support1) + support1